<a href="https://colab.research.google.com/github/simransirsat/MLPracticals/blob/main/NB(6a).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df = pd.read_csv('/content/sample_data/NB.csv', delimiter='\s+')
df_ = df.copy()
print(df)

target_labels = {}
target_labels_list = []

def probab():
    columns = df.columns

    #X_train = df.iloc[0:-test_size, -1]    #
    #y_train = df.iloc[-test_size:, -1]    #Under construction
    #df = df.iloc[]                     #
    
    prob_df = {}
    for i in range(0, df.shape[1] - 1):           # Iterate on columns/attributes:
        if columns[i] not in prob_df:
            prob_df[columns[i]] = {}            # Create a new dataframe
            
        for row in range(0, df.shape[0]):
            if df.iloc[row, i] not in prob_df[columns[i]]:
                prob_df[columns[i]][df.iloc[row, i]] = {}
                prob_df[columns[i]][df.iloc[row, i]][df.iloc[row, -1]] = 1
            elif df.iloc[row, -1] not in prob_df[columns[i]][df.iloc[row, i]]:
                prob_df[columns[i]][df.iloc[row, i]][df.iloc[row, -1]] = 1
            else:
                prob_df[columns[i]][df.iloc[row, i]][df.iloc[row, -1]]+= 1
                

    return prob_df

def get_hashmap_as_df():
    prob_dict = probab()
    #print(prob_dict)

    df_hashmap = {}

    for attr, distribution in prob_dict.items():
        cols = ['name']
        for target, cnt in target_labels.items():
            cols.append(target)
        
        prob_df = pd.DataFrame(columns = cols)
        
        for label, attr_distr in distribution.items():
            temp = attr_distr
            temp['name'] = label
            prob_df = prob_df.append(temp, ignore_index = True)

        df_hashmap[attr] = prob_df.fillna(0)
                   # Not occuring classes position is set to 0,instead of NaN
    return df_hashmap


def probability_distribution(mode = 'basic'):

    df_hashmap = get_hashmap_as_df()

    if mode == 'basic':    
        print('Probability Distributions:\n')

        fractional_df = {}
        for attr, distribution in df_hashmap.items():
            print('\n attr:\t', attr)
            output_df = distribution.copy()
            frac_df = distribution.copy()
            
            for i in range(0, output_df.shape[0]):
                for j in range(1, output_df.shape[1]):
                    frac_df.iloc[i,j] = round(frac_df.iloc[i,j]/target_labels[frac_df.columns[j]],4)

            for i in range(0, output_df.shape[0]):
                for j in range(1, output_df.shape[1]):
                    output_df.iloc[i,j] = str(output_df.iloc[i,j])+'/'+str(target_labels[output_df.columns[j]])+' = '+str(round(output_df.iloc[i,j]/target_labels[output_df.columns[j]],4))
                    
            fractional_df[attr] = frac_df
            fractional_df[attr].set_index('name', inplace = True)
            print(output_df)

        return fractional_df

    elif mode == 'laplacian-smoothing':
        print('Probability Distributions:\n')

        fractional_df = {}
        for attr, distribution in df_hashmap.items():
            print('\n attr:\t', attr)
            output_df = distribution.copy()
            frac_df = distribution.copy()

            c = output_df.shape[0]      # Updated criterion: No. of possible attribute labels
                                        # Previous conflicts: No. of attributes/ No. of target clasess
            for i in range(0, output_df.shape[0]):
                for j in range(1, output_df.shape[1]):
                    frac_df.iloc[i,j] = round((frac_df.iloc[i,j]+1)/(target_labels[frac_df.columns[j]] + c),4)

            for i in range(0, output_df.shape[0]):
                for j in range(1, output_df.shape[1]):
                    output_df.iloc[i,j] = '('+str(output_df.iloc[i,j])+'+ 1)/('+str(target_labels[output_df.columns[j]])+'+'+str(c)+') = '+ str(frac_df.iloc[i,j])
                    
            fractional_df[attr] = frac_df
            fractional_df[attr].set_index('name', inplace = True)
            print(output_df)

            #print('Stored values:\n', fractional_df[attr])

        return fractional_df
        
    elif mode == 'm-estimate':

        print('Probability Distributions:\n')

        fractional_df = {}
        m = 1
        for attr, distribution in df_hashmap.items():
            print('\n attr:\t', attr)
            output_df = distribution.copy()
            frac_df = distribution.copy()
            
            p_inverse = distribution.shape[0]

            for i in range(0, output_df.shape[0]):
                for j in range(1, output_df.shape[1]):
                    frac_df.iloc[i,j] = round((frac_df.iloc[i,j]+m/p_inverse)/(target_labels[frac_df.columns[j]] + m),4)

            for i in range(0, output_df.shape[0]):
                for j in range(1, output_df.shape[1]):
                    output_df.iloc[i,j] = '('+str(output_df.iloc[i,j])+'+'+str(m)+'*1/'+str(p_inverse)+')/('+str(target_labels[output_df.columns[j]])+'+'+str(m)+') = '+str(frac_df.iloc[i,j])
                    
            fractional_df[attr] = frac_df
            fractional_df[attr].set_index('name', inplace = True)
            print(output_df)

        return fractional_df

def predict(model, test_df):
    # model = fractional_df
    assert (test_df.shape[0] == 1), "Send only 1 test case at a time!"

    prob_list = []

    print('\nTest case:\n', test_df)
    for target in target_labels_list:
        print('\nChecking for', target,':\t')
        probability = 1

        print('\t', sep='', end='')
        for i in range(0, test_df.shape[1]):
            print('P(',test_df.iloc[0, i],'|',target,')*',sep='',end='')
        print('P(',target,') = ',sep='')

        print('\t', sep='', end='')
        for i in range(0, test_df.shape[1]):
            print(model[test_df.columns[i]].loc[test_df.iloc[0, i], target],'*', sep='',end='')
        print(round(target_labels[target] / df.shape[0], 4), sep='', end='')

        for i in range(0, test_df.shape[1]):
            probability *= model[test_df.columns[i]].loc[test_df.iloc[0, i], target]
        probability*= round(target_labels[target] / df.shape[0], 4)
        probability = round(probability, 4)
        print(' = ', probability)
        
        
        prob_list.append([round(probability, 4), target])
        

    print(prob_list)
    return max(prob_list, key=lambda x: x[0])[1]

def prediction_center(model, X_test = None, y_test = None):
    df=df_
    if X_test is None:
        X_test = df.iloc[-2:, :-1]          # Test size = 2
        y_test = df.iloc[-2:, -1]

    for i in range(0, X_test.shape[0]):
        classified_as = predict(model, X_test.iloc[i:i+1, :])
        print('Classified as: ', classified_as, '\toriginal: ', y_test.iloc[i])
        if classified_as == y_test.iloc[i]:
            print('Classified correctly')
        else:
            print('Incorrect classification')

def tutorial_mode():
    print('Using m-Estimate:\n\n\t')
    
    global df
    df = df.iloc[0:-2, :]
    print(df)

    global target_labels
    global target_labels_list

    target_labels = {}
    target_labels_list = []

    for i in range(0,df.shape[0]):
        if df.iloc[i,-1] not in target_labels:
            target_labels[df.iloc[i,-1]] = 1
            target_labels_list.append(df.iloc[i, -1])
        else:
            target_labels[df.iloc[i,-1]]+= 1

    prob_dict = probab()
    fractional_df = probability_distribution(mode = 'm-estimate')
    prediction_center(fractional_df)    # Chooses last 2 tuples as test set

    
    df = df_.copy()
    print(df)
    
    target_labels = {}
    target_labels_list = []

    for i in range(0,df.shape[0]):
        if df.iloc[i,-1] not in target_labels:
            target_labels[df.iloc[i,-1]] = 1
            target_labels_list.append(df.iloc[i, -1])
        else:
            target_labels[df.iloc[i,-1]]+= 1

    print('\n\nUsing laplacian-smoothing:\n\n\t')
    fractional_df = probability_distribution(mode = 'laplacian-smoothing')
    print('Test Set: Records 1 and 2')
    prob_dict = probab()
    prediction_center(fractional_df, df.iloc[0:2, :-1], df.iloc[0:2, -1])
    
    print('\n\nUsing laplacian-smoothing:\n\n\t')
    print('Test Set: Records 5 and 6')
    prob_dict = probab()
    prediction_center(fractional_df, df.iloc[5:7, :-1], df.iloc[5:7, -1])

def clear():
    from os import system, name
    if name == 'nt':    # Windows
        _ = system('cls')
    elif name == 'posix':   # Unix:Linux and macos
        _ = system('clear')

#tutorial_mode()
#exit(1)

while True:
    df = df_.copy()

    print('\n Enter suitable option:')
    print('1- Laplacian-Smoothing')
    print('2- Basic Mode')
    print('3- m-Estimate')
    print('4- Clear Screen')
    print('5- Exit')
    print('6- Tutorial Mode')

    choice = int(input(""))
    
    if choice == 1:
        df = df.iloc[0:-2, :]
        print(df)
        target_labels = {}
        target_labels_list = []

        for i in range(0,df.shape[0]):
            if df.iloc[i,-1] not in target_labels:
                target_labels[df.iloc[i,-1]] = 1
                target_labels_list.append(df.iloc[i, -1])
            else:
                target_labels[df.iloc[i,-1]]+= 1        

        print('Using Laplacian-smoothing:\n\n\t')
        fractional_df = probability_distribution(mode = 'laplacian-smoothing')
        prediction_center(fractional_df)

    elif choice == 2:
        df = df.iloc[0:-2, :]
        print(df)
        target_labels = {}
        target_labels_list = []

        for i in range(0,df.shape[0]):
            if df.iloc[i,-1] not in target_labels:
                target_labels[df.iloc[i,-1]] = 1
                target_labels_list.append(df.iloc[i, -1])
            else:
                target_labels[df.iloc[i,-1]]+= 1

        print('Using basic-Mode:\n\n\t')
        fractional_df = probability_distribution(mode = 'basic')
        prediction_center(fractional_df)
    
    elif choice == 3:
        df = df.iloc[0:-2, :]
        print(df)

        target_labels = {}
        target_labels_list = []

        for i in range(0,df.shape[0]):
            if df.iloc[i,-1] not in target_labels:
                target_labels[df.iloc[i,-1]] = 1
                target_labels_list.append(df.iloc[i, -1])
            else:
                target_labels[df.iloc[i,-1]]+= 1

        print('Using m-Estimate:\n\n\t')
        fractional_df = probability_distribution(mode = 'm-estimate')
        prediction_center(fractional_df)

    elif choice == 4:
        clear()
    elif choice == 5:
        exit(1)
    elif choice == 6:
        tutorial_mode()
    else:
        clear()



   Furniture,Rooms,Balcony,Kitchen,Acceptable
0                        Yes,2,2,Moderate,Yes
1                              No,2,2,New,Yes
2                          No,2,3,Moderate,No
3                              Yes,3,2,New,No
4                              Yes,4,3,Old,No
5                             Yes,2,3,New,Yes
6                               No,4,3,Old,No
7                          No,3,2,Moderate,No
8                             Yes,3,3,Old,Yes
9                              Yes,2,3,Old,No
10                             Yes,3,2,New,No
11                        No,3,3,Moderate,Yes
12                             No,3,2,New,Yes
13                             No,2,3,Old,Yes

 Enter suitable option:
1- Laplacian-Smoothing
2- Basic Mode
3- m-Estimate
4- Clear Screen
5- Exit
6- Tutorial Mode
   Furniture,Rooms,Balcony,Kitchen,Acceptable
0                        Yes,2,2,Moderate,Yes
1                              No,2,2,New,Yes
2                          No,2,3,Moderate,No
3        

KeyboardInterrupt: ignored